In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
corpus = ['the food delicious','the food bad','food bad','food delicious','bad']
sent = [1,0,0,1,0] #0: positive , 1:negative

In [3]:
def join_word(text):
    s = ' '.join(word for word in text)
    return(s)

In [4]:
df = pd.DataFrame({'text':corpus,'sent':sent})

In [5]:
df['text'] = df.text.apply(word_tokenize)

In [6]:
df['text'] = df['text'].apply(join_word)

In [7]:
df

,text,sent
0,the food delicious,1
1,the food bad,0
2,food bad,0
3,food delicious,1
4,bad,0


In [8]:
coun_vect = CountVectorizer()
count_matrix = coun_vect.fit_transform(df['text'])
count_array = count_matrix.toarray()
X_countvectorizer = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names())


In [9]:
type(X_countvectorizer)

pandas.core.frame.DataFrame

In [10]:
df_concat = pd.concat([df, X_countvectorizer], axis=1)

In [11]:
df_concat

,text,sent,bad,delicious,food,the
0,the food delicious,1,0,1,1,1
1,the food bad,0,1,0,1,1
2,food bad,0,1,0,1,0
3,food delicious,1,0,1,1,0
4,bad,0,1,0,0,0


In [12]:
df_concat.drop('text',axis = 1,inplace = True)

In [13]:
df = df_concat 

In [14]:
df

,sent,bad,delicious,food,the
0,1,0,1,1,1
1,0,1,0,1,1
2,0,1,0,1,0
3,1,0,1,1,0
4,0,1,0,0,0


In [15]:
# adding dummy rows to avoid 0 in probability
#df.loc[len(df.index)] = [0,1,1,1,1] 
#df.loc[len(df.index)] = [1,1,1,1,1] 

In [16]:
df

,sent,bad,delicious,food,the
0,1,0,1,1,1
1,0,1,0,1,1
2,0,1,0,1,0
3,1,0,1,1,0
4,0,1,0,0,0
5,0,1,1,1,1
6,1,1,1,1,1


In [17]:
to_predict = ['the','food','delicious']

In [27]:
def naive_bayes(data,text):
    pred = {}
    #pred["i"] = (probability)

    for i in data.sent.unique():
        probability = 1
        count_sent = data['sent'].value_counts().to_dict()
        probability *= (count_sent[i]/len(data))
        total = data.sum().to_dict()
        target = data[data['sent'] == i].sum().to_dict()
        for text in to_predict:
            probability *= (target[text]/total[text])
        pred[i] = (probability)
        #print (i,probability)

    values = sum(pred.values())
    for p in pred:
        print('the proability of belonging to class %d  is %f'  %(p , pred[p]/values))

In [28]:
naive_bayes(df,to_predict)

the proability of belonging to class 1  is 0.692308
the proability of belonging to class 0  is 0.307692
